In [2]:
import requests
import pandas as pd
from collections import OrderedDict

#здесь 3 запроса SPARQL к Викиданным, которые выдают список списков с высказываниями с датами. 
#один общий запрос, скорее всего, невозможен, поскольку одной строке в табличке (=одному высказыванию) 
#может соответствовать только одна дата (или две - если это дата начала и дата окончания чего-то).
#если в одном высказывании будет много дат, то будет трудно разобраться, что к чему


#http://tinyurl.com/y9m738lr  - простые триплеты с датой
def query_for_simple_triplets(q_numbers):
    
    url = "https://query.wikidata.org/sparql"
    
    query = """SELECT ?itemLabel ?propertyLabel ?date
    WHERE
    {
      VALUES (?item) {(wd:Q???)}
  
      ?item ?predicate ?date. 
      FILTER(DATATYPE(?date) = xsd:dateTime).
      MINUS { ?item schema:dateModified ?date. }
      MINUS { ?item wikibase:timestamp ?date. }
  
      BIND( IRI(REPLACE( STR(?predicate),"prop/direct/","entity/" )) AS ?property). 
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],ru". }
    }
    ORDER BY ?item ?date
    """
        
    query = query.replace('(wd:Q???)',q_numbers)
    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()
    
    lst = []
    if data != None:
        for item in data['results']['bindings']:
            lst.append([item['itemLabel']['value'], 
                        item['propertyLabel']['value'], 
                        str(item['date']['value'])[:-10]]) #"обрезание" даты. потом вставлю регулярку
    return lst


#http://tinyurl.com/yboysd2z  - start time-end time
def query_for_starttime_endtime(q_numbers):
    
    url = "https://query.wikidata.org/sparql"
    
    query = """SELECT ?itemLabel ?propertyLabel ?objectLabel ?organizationLabel ?starttime 
    ?endtime
    WHERE {
      VALUES (?item) {(wd:Q???)}
  
      ?item ?predicate ?statement .  
      ?statement ?ps ?object. 
      OPTIONAL {?statement pq:P642 ?organization}.
      ?statement pq:P580 ?starttime.
      OPTIONAL {?statement pq:P582 ?endtime}. 
   
      ?property wikibase:statementProperty ?ps .
  
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],ru" }
    }
    ORDER BY ?itemLabel ?propertyLabel
    """
    
    query = query.replace('(wd:Q???)',q_numbers)
    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()
    
    lst = []
    for item in data['results']['bindings']:    
        lst.append([item['itemLabel']['value'], 
                    item['propertyLabel']['value'],
                    item['objectLabel']['value'],
                    item['organizationLabel']['value'] if 'organizationLabel' in item else '',
                    str(item['starttime']['value'])[:-10],
                    str(item['endtime']['value'])[:-10]] if 'endtime' in item else '')

    return lst


#http://tinyurl.com/y947qncg  -　point in time
def query_for_point_in_time(q_numbers):
    
    url = "https://query.wikidata.org/sparql"
    
    query = """SELECT ?itemLabel ?propertyLabel ?objectLabel ?organizationLabel ?point_in_time
    WHERE {
      VALUES (?item) {(wd:Q???)}
  
      ?item ?predicate ?statement .  
      ?statement ?ps ?object. 
      OPTIONAL {?statement pq:P642 ?organization}.
      ?statement pq:P585 ?point_in_time.
   
      ?property wikibase:statementProperty ?ps .
  
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],ru" }
    }
    ORDER BY ?itemLabel ?propertyLabel
    """
    
    query = query.replace('(wd:Q???)',q_numbers)
    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()
    
    lst = []
    for item in data['results']['bindings']:    
        lst.append([item['itemLabel']['value'], 
                    item['propertyLabel']['value'],
                    item['objectLabel']['value'],
                    item['organizationLabel']['value'] if 'organizationLabel' in item else '',
                    str(item['point_in_time']['value'])[:-10]])

    return lst

q_numbers = '(wd:Q635)(wd:Q7747)(wd:Q243)'#здесь должны быть номера объектов из наших n-грамм
#в Викиданных, которые мы подставляем в запрос

dates_list = query_for_simple_triplets(q_numbers) + query_for_starttime_endtime(q_numbers) + query_for_point_in_time(q_numbers)
for i in dates_list:
    if i != '':
        print(i)

['Эйфелева башня', 'дата основания/создания/возникновения', '1887-01-01']
['Эйфелева башня', 'дата официального открытия', '1889-03-31']
['Эйфелева башня', 'дата перехода в общественное достояние', '1993-01-01']
['Клеопатра', 'дата рождения', '-068-01-01']
['Клеопатра', 'дата смерти', '-029-08-10']
['Владимир Владимирович Путин', 'дата рождения', '1952-10-07']
['Владимир Владимирович Путин', 'вид или род войск', 'Комитет государственной безопасности СССР', '', '1975-01-01', '1991-01-01']
['Владимир Владимирович Путин', 'владеет', 'ядерный чемоданчик (Россия)', '', '1999-12-31', '2008-05-07']
['Владимир Владимирович Путин', 'занимаемая должность', 'председатель Правительства Российской Федерации', '', '1999-08-16', '2000-05-07']
['Владимир Владимирович Путин', 'занимаемая должность', 'руководство ФСБ России', '', '1998-07-25', '1999-03-29']
['Владимир Владимирович Путин', 'занимаемая должность', 'Временное исполнение полномочий президента России', '', '1999-12-31', '2000-05-07']
['Влади

In [5]:
#примерно так по году можно выбирать только релевантные высказывания
date = '1999'
for statement in dates_list:
    if statement != '' and (statement[-1].startswith(date) or statement[-2].startswith(date)): 
        print(statement)

['Владимир Владимирович Путин', 'владеет', 'ядерный чемоданчик (Россия)', '', '1999-12-31', '2008-05-07']
['Владимир Владимирович Путин', 'занимаемая должность', 'председатель Правительства Российской Федерации', '', '1999-08-16', '2000-05-07']
['Владимир Владимирович Путин', 'занимаемая должность', 'руководство ФСБ России', '', '1998-07-25', '1999-03-29']
['Владимир Владимирович Путин', 'занимаемая должность', 'Временное исполнение полномочий президента России', '', '1999-12-31', '2000-05-07']
['Владимир Владимирович Путин', 'занимаемая должность', 'секретарь Совета безопасности Российской Федерации', '', '1999-03-29', '1999-08-09']
['Владимир Владимирович Путин', 'занимаемая должность', 'Верховный Главнокомандующий Вооружёнными Силами Российской Федерации', '', '1999-12-31', '2008-05-07']
['Владимир Владимирович Путин', 'занимаемая должность', 'Временное исполнение полномочий президента России', '', '1999-12-31', '2000-05-07']
['Владимир Владимирович Путин', 'питомец', 'Конни Полгрей

In [7]:
#потом потестирую формат вывода датафрейм. пока обычные списки кажутся удобнее
#это результат запроса простых триплетов с датами

url = "https://query.wikidata.org/sparql"
    
query = """SELECT ?itemLabel ?propertyLabel ?date
    WHERE
    {
      VALUES (?item) {(wd:Q???)}
  
      ?item ?predicate ?date. 
      FILTER(DATATYPE(?date) = xsd:dateTime).
      MINUS { ?item schema:dateModified ?date. }
      MINUS { ?item wikibase:timestamp ?date. }
  
      BIND( IRI(REPLACE( STR(?predicate),"prop/direct/","entity/" )) AS ?property). 
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],ru". }
    }
    ORDER BY ?item ?date
    """

q_numbers = '(wd:Q635)(wd:Q7747)(wd:Q243)'
query = query.replace('(wd:Q???)',q_numbers)
r = requests.get(url, params = {'format': 'json', 'query': query})
data = r.json()
    
for item in data['results']['bindings']:
    dates.append(OrderedDict({
        'item': item['itemLabel']['value'],
        'property': item['propertyLabel']['value'],
        'date': str(item['date']['value'])[:-10]})) 

df = pd.DataFrame(dates)
df.set_index('item', inplace=True)
df = df.astype({'property': str, 'date': str})
df.head()

,property,date
item,,
Эйфелева башня,дата основания/создания/возникновения,1887-01-01
Эйфелева башня,дата официального открытия,1889-03-31
Эйфелева башня,дата перехода в общественное достояние,1993-01-01
Клеопатра,дата рождения,-068-01-01
Клеопатра,дата смерти,-029-08-10
